In [1]:
library(dplyr)
library(DBI)
library(stringr)
library(ggplot2)
library(reticulate)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
### getting database connection, fetching tables ###

getDBConnection <- function() {
    # returns a connection object to fetch db tables 

    conn <- DBI::dbConnect(
                RPostgres::Postgres(), 
                dbname = "fred", 
                host = Sys.getenv("PGHOST"), 
                port = Sys.getenv("PGPORT"), 
                user = Sys.getenv("PGUSER"), 
                password = Sys.getenv("PGPW"))
    return (conn)
}

fetchTable <- function(table_name) {
    # uses connection object to fetch and return tables stored in db 

    if (str_detect(table_name, " ")) {
        stop("Table name does not include whitespace")
        return (1) # exit code 1 on exception 
    }
    conn <- getDBConnection()
    query <- conn %>% DBI::dbSendQuery(
            paste("select * from ", table_name))
    data <- query %>% DBI::dbFetch() %>% as_tibble()
    query %>% DBI::dbClearResult()
    conn %>% DBI::dbDisconnect()
    return (data)
}


In [3]:
gdp <- "real_gross_domestic_product" %>% fetchTable()
hpi <- "s_p_case_shiller_u_s__national_home_price_index" %>% fetchTable()
reit <- "wilshire_us_real_estate_investment_trust_total_market_index__wi" %>% fetchTable()
sahm <- "real_time_sahm_rule_recession_indicator" %>% fetchTable()
tpd_pgdp <- "federal_debt__total_public_debt_as_percent_of_gross_domestic_product_pr" %>% fetchTable()
tfr <- "total_reserve_balances_maintained_With_federal_reserve_banks__d" %>%fetchTable()


In [4]:
fredapi <- import('fredapi')
fredapi %>% names()
Fred <- fredapi$Fred(api_key = Sys.getenv("FRED_API_KEY"))
Fred %>% names()

getFredMetadata <- function(df) {
    # input: a tibble or dataframe object

    # returns: 
    # metadata from Fred API via Python fredapi interface stored in 
    # a tibble and with each rowname stored in the surrogate rowname 
    # column called 'name'
    # tibbles don't store rownames so rownames are stored in a column 
    # this function relies on reticulate R interface to Python

    # the official name of the series is stored at the third index of 
    # the vector returned by colnames(df) for all tables 
    # first two columns are timepoint_id and date_of_obs
    series_info <- Fred$get_series_info(colnames(df)[3]) 
    series_info_names <- series_info %>% 
                            lapply(names) %>% 
                            attributes() %>% 
                            as_tibble()
    metadata_tbl <- series_info %>% 
                        as_tibble() %>% 
                        bind_cols(series_info_names) %>% 
                        select(names, value)
    return (metadata_tbl)
}

gdp_metadata <- gdp %>% getFredMetadata()
hpi_metadata <- hpi %>% getFredMetadata()
reit_metadata <- reit %>% getFredMetadata()
sahm_metadata <- sahm %>% getFredMetadata()
tpd_pgdp_metadata <- tpd_pgdp %>% getFredMetadata()
tfr_metadata <- tfr %>% getFredMetadata()


[1] "fred"    "Fred"    "version"

[1] "api_key"                   "earliest_realtime_start"  
 [3] "get_series"                "get_series_all_releases"  
 [5] "get_series_as_of_date"     "get_series_first_release" 
 [7] "get_series_info"           "get_series_latest_release"
 [9] "get_series_vintage_dates"  "latest_realtime_end"      
[11] "max_results_per_request"   "nan_char"                 
[13] "root_url"                  "search"                   
[15] "search_by_category"        "search_by_release"

In [5]:
### data description / notes from fredapi 

In [6]:

getMetadataVals <- function(df_metadata, data_point) {
    # input: 
    # df_metadata: a Fred API metadata tibble 
    # data_point: a string denoting the row name whose value to return 
    # returns: a metadata element such as title or frequency 
    # from fredapi metadata tibble 

    metadata_element <- df_metadata %>% 
                            filter(names == data_point) %>% 
                            pull(value)
                            #select(value) %>% as.character()
    return (metadata_element)
}



In [7]:
notes_gdp <- gdp_metadata %>% getMetadataVals('notes')
notes_gdp

[1] "BEA Account Code: A191RL  Gross domestic product (GDP) is the value of the goods and services produced by the nation's economy less the value of the goods and services used up in production. GDP is also equal to the sum of personal consumption expenditures, gross private domestic investment, net exports of goods and services, and government consumption expenditures and gross investment. Real values are inflation-adjusted estimates—that is, estimates that exclude the effects of price changes.  For more information about this series, please visit the Bureau of Economic Analysis (http://www.bea.gov/national/)."

In [8]:
notes_hpi <- hpi_metadata %>% getMetadataVals('notes')  
notes_hpi

[1] "For more information regarding the index, please visit Standard & Poor's (https://www.spglobal.com/spdji/en/documents/methodologies/methodology-sp-corelogic-cs-home-price-indices.pdf). There is more information about home price sales pairs in the Methodology section. Copyright, 2016, Standard & Poor's Financial Services LLC. Reprinted with permission."

In [9]:
notes_reit <- reit_metadata %>% getMetadataVals('notes') 
notes_reit

[1] "The observations for the Wilshire US Real Estate Investment Trust Total Market Index (Wilshire US REIT) represent the daily index value at market close. The market typically closes at 4 PM ET, except for holidays when it sometimes closes early.  The total market indexes are total market returns, which do include reinvested dividends. Copyright, 2016, Wilshire Associates Incorporated. Reprinted with permission. For more information about the various indexes, visit Wilshire Associates (http://www.wilshire.com/Indexes)."

In [10]:
notes_sahm <- sahm_metadata %>% getMetadataVals('notes')
notes_sahm

[1] "Sahm Recession Indicator signals the start of a recession when the three-month moving average of the national unemployment rate (U3) (https://fred.stlouisfed.org/series/UNRATE) rises by 0.50 percentage points or more relative to its low during the previous 12 months.  This indicator is based on \"real-time\" data, that is, the unemployment rate (and the recent history of unemployment rates) that were available in a given month. The BLS revises the unemployment rate each year at the beginning of January, when the December unemployment rate for the prior year is published. Revisions to the seasonal factors can affect estimates in recent years. Otherwise the unemployment rate does not revise."

In [11]:
notes_tpd <- tpd_pgdp_metadata %>% getMetadataVals('notes')
notes_tpd 

[1] "Federal Debt: Total Public Debt as Percent of Gross Domestic Product (GFDEGDQ188S) was first constructed by the Federal Reserve Bank of St. Louis in October 2012. It is calculated using Federal Government Debt: Total Public Debt (GFDEBTN) and Gross Domestic Product, 1 Decimal (GDP): GFDEGDQ188S = ((GFDEBTN/1000)/GDP)*100 GFDEBTN/1000 transforms GFDEBTN from millions of dollars to billions of dollars."

In [12]:
notes_tfr <- tfr_metadata %>% getMetadataVals('notes')
notes_tfr

[1] "The Board of Governors discontinued the H.3 statistical release on September 17, 2020. For more information, please see the announcement (https://www.federalreserve.gov/feeds/h3.html) posted on August 20, 2020.  Total reserve balances maintained is the amount of balances institutions hold in accounts at Federal Reserve Banks that are available to satisfy reserve requirements. Historically, this series excluded balances held in a reserve account for contractual clearing purposes (contractual clearing balances program has been discontinued on July 12, 2012).  Effective February 2, 1984, reserve computation and maintenance periods have been changed from weekly to bi-weekly. Series with data prior to February 2, 1984 have different values reported from one week to the next. After February 2, 1984, the value repeats for 2 consecutive weeks."

In [13]:
### cleaning and interpolating between known-values for tfr:: fed reserves held 

In [14]:

# drops NA and extra id column
# changes only inconvenient column names
gdp <- gdp %>% 
        rename(pct_ch_gdp = a191rl1q225sbea) %>%
        select(date_of_obs, pct_ch_gdp)

hpi <- hpi %>% 
            rename(hp_index = csushpisa) %>%
            filter(!is.na(hp_index)) %>%
            select(date_of_obs, hp_index)

reit <- reit %>% 
            filter(!is.na(willreitind)) %>% 
            select(date_of_obs, willreitind)

sahm <- sahm %>% select(date_of_obs, sahmrealtime)


tpd_pgdp <- tpd_pgdp %>% 
                rename(tpd_pctgdp = gfdegdq188s) %>%
                select(date_of_obs, tpd_pctgdp)

tfr <- tfr %>% 
            rename(tot_res = resbalnsw) %>%
            filter(!is.na(tot_res)) %>%
            select(date_of_obs, tot_res)

In [15]:

getDifferencedSeries <- function(df, col_string, difference = 1) {
    # returns series' change from preceding period by default 

    # assigns the number of differences to calculate
    diffs_left_to_calculate <- difference
    # not feasible to difference a vector while it's embedded 
    # in a tibble, so it's passed to matrix constructor, 
    # differenced, then become a vector 
    differenced <- df %>% select(all_of(col_string)) %>% 
                as.matrix() %>% diff() %>% as.vector()
    diffs_left_to_calculate <- diffs_left_to_calculate - 1

    # stopping condition is true when there are no more 
    # differences to calculate 
    while (diffs_left_to_calculate > 0) {
        differenced <- differenced %>% diff()
        diffs_left_to_calculate <- diffs_left_to_calculate - 1
    }
    return (differenced)
}


In [16]:

setDifferencedTibble <- function(df, diff_vector, col_string, retain = TRUE) {
    # retains undifferenced series and adds differenced version
    # unless retain is set to false

    # adds differenced series 
    new_df <- df[-1, ] %>% mutate(series_1d = diff_vector) 

    if (!retain) {     
        # reorders columns
        new_df <- new_df %>% select(date_of_obs, series_1d)
        return (new_df)
    }
    new_df <- new_df %>% select(date_of_obs, series_1d, all_of(col_string))
    return (new_df)
}


In [17]:

hpi <- hpi %>% 
            getDifferencedSeries("hp_index") %>% 
            setDifferencedTibble(
                df = hpi,
                col_string = "hp_index", FALSE) %>%
            rename(hp_index = series_1d)

reit <- reit %>%
            getDifferencedSeries("willreitind") %>%
            setDifferencedTibble(
                df = reit,
                col_string = "willreitind", FALSE) %>%
            rename(willreitind = series_1d) 

tpd_pgdp <- tpd_pgdp %>%
            getDifferencedSeries("tpd_pctgdp") %>%
            setDifferencedTibble(
                df = tpd_pgdp,
                col_string = "tpd_pctgdp", FALSE) %>%
            rename(tpd_pctgdp = series_1d)

tfr <- tfr %>%
            getDifferencedSeries("tot_res") %>%
            setDifferencedTibble(
                df = tfr,
                col_string = "tot_res", FALSE) %>%
            rename(tot_res = series_1d) 

In [18]:
tfr %>% tail()

date_of_obs,tot_res
<date>,<dbl>
2020-08-05,127598
2020-08-12,0
2020-08-19,70616
2020-08-26,0
2020-09-02,32950
2020-09-09,0


In [19]:
# interpolation for post-frequency change values in total federal reserve balances  

# post_change is the vector of all tot_res values recorded after 
# the 1984-February-02 change in recording frequency 
post_change <- tfr %>% filter(date_of_obs >= '1984-02-02') %>% pull(tot_res)

# filled_cells stores all cells / only cells that have non-duplicate 
# values. these cells are non-zero but not necessarily 
# more specifically they're all cells of post_change that are not 
# 0 by default 
filled_cells <- post_change[seq(1, length(post_change), by = 2)]

interpolated_tot_res <- filled_cells[1:length(filled_cells)] %>% approx(n = 2 * length(filled_cells) - 1)
# has x, y but x isn't relevant and will prevent mutate function from working 
interpolated_tot_res <- interpolated_tot_res$y

interpolationQA <- function(value1, value2) {
    # returns interpolated value between adjacent value1 and value2 
    # quality assurance 
    # interpolation formula: Xi-1 + (.5 * (Xi - Xi-1)) 
    return( value2 + (.5 * (value1 - value2)))
}

interpolationQA(filled_cells[3], filled_cells[4])

tfr_post_change <- tfr %>% 
    filter(date_of_obs >= '1984-02-02') %>% 
    mutate(missing = 0)
tfr_post_change_missing <- tfr_post_change[seq(2, nrow(tfr_post_change), by = 2), ] %>% mutate(missing = 1)
tfr_post_change_present <- tfr_post_change[seq(1, nrow(tfr_post_change), by = 2), ]

# assurance that missing rows and present rows were separated 
setdiff(tfr_post_change_missing, tfr_post_change_present) == tfr_post_change_missing

# if last value is repeated and not unique then trimming is needed i.e., if 
# last row of first-differenced tibble holds value of 0 


if (tfr_post_change[nrow(tfr_post_change), "tot_res"] == 0) {
tfr_post_change <- tfr_post_change[-nrow(tfr_post_change), ]
}

tfr_post_change <- tfr_post_change %>% 
                        mutate(tot_res = interpolated_tot_res) %>% 
                        select(date_of_obs, tot_res)

# the setdiff function returns pre-1984-February-02 dates: dates for which no 
# interpolation is necessary and these rows are bound to tfr after 
# 1984-February-02 with missing values interpolated
tfr <- setdiff(tfr, tfr_post_change) %>% bind_rows(tfr_post_change)


[1] 324.5

date_of_obs,tot_res,missing
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE
TRUE,TRUE,TRUE


In [20]:
tfr %>% tail()

date_of_obs,tot_res
<date>,<dbl>
2020-07-29,-22330.5
2020-08-05,127598.0
2020-08-12,99107.0
2020-08-19,70616.0
2020-08-26,51783.0
2020-09-02,32950.0


In [21]:
### getting summary statistics 

In [22]:

# kurtosis & skewness 
getMoments <- function(df, col_string, moment) {
    # returns skewness or kurtosis of the series
    # ---------
    # kurtosis: 
    # uses Fisher's method  
    # subtracts 3 to report only excess kurtosis
    # normal distribution has Kurtosis = 0
    # skewness: 
    # uses Pearson's method

    u_scaled <- df %>% 
                pull(all_of(col_string)) %>% 
                scale()
    skewness_or_K <- u_scaled ** moment %>% sum()
    # adjust with n - 1
    skewness_or_K <- skewness_or_K / (length(u_scaled) - 1)

    # return without subtraction if returning skewness
    if (moment == 3) {
        return (skewness_or_K)
    }
    # return kurtosis as only excess kurtosis  
    return (skewness_or_K - 3)
}

getSumStats <- function(df, col_string) {
    # focus is the column that's being summarised 
    # it's renamed so that this function can be generic / viable for more than 1 tibble  

    skewness <- df %>% getMoments(col_string = col_string, moment = 3)
    kurtosis <- df %>% getMoments(col_string = col_string, moment = 4)
    sum_stats <- df %>% 
                    rename(focus = all_of(col_string)) %>% 
                    summarise(
                        quantiles = t(quantile(focus)),
                        mean = mean(focus), 
                        sd = sd(focus),
                        mad = mad(focus),
                        var = var(focus)) %>% 
                    t() %>% 
                    as.data.frame() %>% 
                    tibble::rownames_to_column() %>% 
                    as_tibble() %>% 
                    rename(focus = V1) %>% 
                    add_row(rowname = 'skewness', focus = skewness) %>% 
                    add_row(rowname = 'kurtosis', focus = kurtosis)

    return (sum_stats)
}


In [23]:
gdp_sum_stats <- gdp %>% getSumStats(col_string = 'pct_ch_gdp') %>% rename(pct_ch_gdp = focus)
gdp_sum_stats

rowname,pct_ch_gdp
<chr>,<dbl>
quantiles.0%,-31.400000
quantiles.25%,1.200000
quantiles.50%,3.000000
quantiles.75%,5.100000
quantiles.100%,16.700000
mean,3.061775
sd,4.348497
mad,2.816940
var,18.909424


In [24]:
hpi_sum_stats <- hpi %>% getSumStats(col_string = 'hp_index') %>% rename(hp_index = focus)
hpi_sum_stats

rowname,hp_index
<chr>,<dbl>
quantiles.0%,-2.5230000
quantiles.25%,0.1052500
quantiles.50%,0.4345000
quantiles.75%,0.7945000
quantiles.100%,2.4690000
mean,0.3856095
sd,0.7347630
mad,0.5144622
var,0.5398767


In [25]:
reit_sum_stats <- reit %>% getSumStats(col_string = 'willreitind') %>% rename(willreitind = focus)
reit_sum_stats

rowname,willreitind
<chr>,<dbl>
quantiles.0%,-1807.800000
quantiles.25%,-18.590000
quantiles.50%,2.000000
quantiles.75%,27.420000
quantiles.100%,796.700000
mean,1.563143
sd,85.173816
mad,34.322190
var,7254.578976


In [26]:
sahm_sum_stats <- sahm %>% getSumStats(col_string = 'sahmrealtime') %>% rename(sahmrealtime = focus)
sahm_sum_stats

rowname,sahmrealtime
<chr>,<dbl>
quantiles.0%,-0.3700000
quantiles.25%,-0.0300000
quantiles.50%,0.0300000
quantiles.75%,0.4300000
quantiles.100%,9.5000000
mean,0.4239452
sd,0.9695381
mad,0.1927380
var,0.9400042


In [27]:
tpd_pgdp_sum_stats <- tpd_pgdp %>% getSumStats(col_string = 'tpd_pctgdp') %>% rename(tpd_pctgdp = focus)
tpd_pgdp_sum_stats

rowname,tpd_pctgdp
<chr>,<dbl>
quantiles.0%,-2.2323000
quantiles.25%,-0.3183600
quantiles.50%,0.2550300
quantiles.75%,0.7981400
quantiles.100%,27.9293700
mean,0.4391743
sd,2.1435156
mad,0.8354599
var,4.5946589


In [28]:
tfr_sum_stats <- tfr %>% getSumStats(col_string = 'tot_res') %>% rename(tot_res = focus)
tfr_sum_stats

rowname,tot_res
<chr>,<dbl>
quantiles.0%,-2.581200e+05
quantiles.25%,-4.920000e+02
quantiles.50%,0.000000e+00
quantiles.75%,5.490000e+02
quantiles.100%,6.059000e+05
mean,1.691135e+03
sd,3.277719e+04
mad,7.605738e+02
var,1.074344e+09
